In [1]:
import pandas as pd
import csv, json, glob
from pathlib import Path
import numpy as np

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

### Helpers 

In [3]:
def get_files(path, pattern):
    return [x for x in list(Path(path).glob(pattern))]

In [4]:
def get_dataframes_from_csv(files, custom_names):
    return [pd.read_csv(x, names=custom_names) for x in files]

def mean_time_per_trial(trials):
    return np.array([len(x['time']) for x in trials])

def mean_power_per_trial(trials):
    return np.array([x['power'].mean() for x in trials])

def get_energy_per_trial(trials):
    out = []
    for x in trials:
        en = np.trapz(x['power'], x=x['time'])
        out.append(en)
    return out

""""def total_energy_experiment(trials):
        return np.array([np.trapz(x['power'], x['time']) for x in trials])
"""

'"def total_energy_experiment(trials):\n        return np.array([np.trapz(x[\'power\'], x[\'time\']) for x in trials])\n'

In [5]:
def retrieve_performance_stats(files):
    res = []
    for x in files:
        cmd = !python3.10 ./scripts/show_stats.py -f "{x}" # calculate stats
        res.append(cmd)
    return res

def get_performance_dataframes(stats):
    return [[[float(x) for x in values.split()[1:]] for values in run] for run in stats]

In [6]:
def plot_dataframes(dfs, cols, rows):
    plt.rcParams["figure.figsize"] = (50, 50)
    fig, axes = plt.subplots(cols, rows)

    cnt = 0
    for x in range(0, cols):
        for y in range(0, rows):
            dfs[cnt]['power'].plot.line(ax=axes[x, y])
            cnt += 1

In [7]:
def get_power_interval(df):
    x = next((i for i,v in enumerate(df['power']) if v >= 50), None)
    y = next((i for i,v in enumerate(reversed(df['power'])) if v >= 50), None)

    return df[x:(df.index[-1] - y)]

In [8]:
def get_stats(dfs):
    time   = np.array(mean_time_per_trial(dfs))
    #energy = np.array(get_energy_per_trial(dfs))
    power = mean_power_per_trial(dfs)
    energy = power * np.array([len(df['time']) for df in dfs])
    
    
    e = energy/time
    
    print(
        pd.DataFrame({'energy': energy, 'time': time, 'e': energy/time}), '\n'
    )

    print(
            f"# Energy Per Visit(Joule/Visit):\n",
            f"## Mean:\t\t\t{energy.mean()}", 
            f"## Min-Max:\t\t\t[{energy.min()}, {energy.max()}]",
            f"## Var:\t\t\t\t{energy.var()}", 
            f"## Std:\t\t\t\t{energy.std()}", 
            '\n'
            f"# Average Response Time:\t{time.mean()}",
            f"# e (Joule/s):\t\t\t{e.mean()}, [{e.min()}, {e.max()}]",
            sep='\n'
         )

    return energy, time, e

In [9]:
path = "./delayed-30/"
cust = 150

### Energy Consumption

In [10]:
files  = get_files(f"{path}", f"*-{cust}-energy.csv")
dfs    = get_dataframes_from_csv(files,["time", "power"])
dfs = [get_power_interval(df) for df in dfs]
#dfs = [df[2:9] for df in dfs]
energy, time, e = get_stats(dfs)

Empty DataFrame
Columns: [energy, time, e]
Index: [] 



/tmp/ipykernel_106242/3418792219.py:16: RuntimeWarning: Mean of empty slice.
  f"## Mean:\t\t\t{energy.mean()}",
/home/xeder/miniconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ValueError: zero-size array to reduction operation minimum which has no identity

In [ ]:
#plot_dataframes(dfs, 5, 6)

In [ ]:
files = get_files(f"{path}", f"*-{cust}-perf.json")
stats = retrieve_performance_stats(files)
dfs   = [pd.DataFrame(x) for x in get_performance_dataframes(stats)]
dfs = [df.head(5) for df in dfs] # remove first and last row
#pd.DataFrame({i:df[0] for i, df in enumerate(dfs)})
u = np.array([df[0][1:].mean() for i, df in enumerate(dfs)])

In [ ]:
print(u.min(), u.max())